# Preparing the enviroment and mounting data

In [1]:
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
#from google.colab import drive
#drive.mount("/content/drive/")
#cd drive/My Drive/SFU/Project4-Spring2021
!ls

clinical_01Mar2021.xlsx        sample_data		 snpsback_variants.npy
data_preprossing_functions.py  samples.npy		 x_clinical.npy
known_snps_fullname.txt        samples_order.txt	 x_colnames.npy
known_snps.txt		       snpsback_cadd.npy	 x_snps.npy
__pycache__		       snpsback_gt_dominant.npz  y.npy


# BCCH Data Analysis 

## 1. Data Pre-processing 

### Reading the files 



In [2]:
# reading snps file
#libraries
import pandas as pd
import numpy as np 
from scipy import sparse


In [7]:
preprocessing = False
path = '/content/'
if preprocessing: 
  print('Starting pre-processing:')
  import data_preprossing_functions as dpf
  y, x_clinical, x_snps, x_colnames = dpf.run_preprocessing(path, True)
  print('Done!')
  print('Shapes:',len(y), x_clinical.shape, x_snps.shape, len(x_colnames))
  #318 (318, 42) (318, 16641) 16641
else: 
  print('Reading files')
  y, x_clinical, x_snps, x_colnames = np.load('y.npy'), np.load('x_clinical.npy',allow_pickle =True), np.load('x_snps.npy',allow_pickle =True), np.load('x_colnames.npy',allow_pickle =True)
  print('Done!')
  print('Shapes:',len(y), x_clinical.shape, x_snps.shape, len(x_colnames))
 

Reading files
/Done!
Shapes: 318 (318, 42) (318, 16641) 16641
